# [2.X코드반영] Lab-11-4: mnist cnn keras ensemble eager


## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.0


## Hyper Parameters

In [2]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
tf.random.set_seed(777)

## Creating a Checkpoint Directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_ensemble'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## MNIST Data

In [4]:
mnist = keras.datasets.mnist
class_names = ['0','1','2','3','4','5','6','7','8','9']

## Datasets

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.astype(np.float32)/255.
test_images = test_images.astype(np.float32)/255.

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size = 1000000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

11493376/11490434 [==============================] - 0s 0us/step


## Model Class

In [6]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.pool1 = keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.pool2 = keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME')
        self.pool3 = keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = keras.layers.Flatten()
        self.dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10)
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [7]:
models = []
num_models = 3
for m in range(num_models):
  models.append(MNISTModel())

## Loss Function

In [8]:
def loss_fn(model, images, labels):
  logits = model(images, training=True)
  loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
      y_pred=logits, y_true=labels, from_logits=True))
  return loss

## Calculating Gradient

In [9]:
def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)

## Caculating Model's Accuracy

In [10]:
def evaluate(models, images, labels):
  predictions = np.zeros_like(labels)
  for model in models:
    logits = model(images, training=False)
    # 단순 가산
    predictions += logits
  correct_prediction = tf.equal(tf.argmax(predictions,1), tf.argmax(labels, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

## Optimizer

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

## Creating a Checkpoint

In [12]:
# 모델별로 체크포인트 저장
checkpoints = []
for m in range(num_models):
  checkpoints.append(tf.train.Checkpoint(cnn=models[m]))

## Training

In [14]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0    
    
    for images, labels in train_dataset:
        for model in models:
            #train(model, images, labels)
            grads = grad(model, images, labels)                
            optimizer.apply_gradients(zip(grads, model.variables))
            loss = loss_fn(model, images, labels)
            avg_loss += loss / num_models
        acc = evaluate(models, images, labels)
        avg_train_acc += acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(models, images, labels)        
        avg_test_acc += acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    # 각각 모델의 체크포인트 저장
    for idx, checkpoint in enumerate(checkpoints):
        checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.04023503 train accuracy =  0.9930 test accuracy =  0.9934
Epoch: 2 loss = 0.02688042 train accuracy =  0.9957 test accuracy =  0.9931
Epoch: 3 loss = 0.01966896 train accuracy =  0.9971 test accuracy =  0.9931
Epoch: 4 loss = 0.01586931 train accuracy =  0.9983 test accuracy =  0.9940
Epoch: 5 loss = 0.01269333 train accuracy =  0.9988 test accuracy =  0.9941
Epoch: 6 loss = 0.01032155 train accuracy =  0.9990 test accuracy =  0.9946
Epoch: 7 loss = 0.00966371 train accuracy =  0.9994 test accuracy =  0.9927
Epoch: 8 loss = 0.00834729 train accuracy =  0.9994 test accuracy =  0.9947
Epoch: 9 loss = 0.00679439 train accuracy =  0.9997 test accuracy =  0.9948
Epoch: 10 loss = 0.00631700 train accuracy =  0.9998 test accuracy =  0.9948
Epoch: 11 loss = 0.00542638 train accuracy =  0.9997 test accuracy =  0.9948
Epoch: 12 loss = 0.00554531 train accuracy =  0.9998 test accuracy =  0.9945
Epoch: 13 loss = 0.00465642 train accuracy =  0.